**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/joining-data).**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [4]:
#table = dataset.table

# Get a list of available tables 
list_of_tables_array = []
list_of_tables = client.list_tables(dataset) # Your code here
for table in list_of_tables:
    print(table.table_id)
    list_of_tables_array.append(table.table_id)
list_of_tables = list_of_tables_array
# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes
['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
#q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [6]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,63172172,None,<p>Up to now 7 answers. But I am really surpri...,None,None,0,None,2020-07-30 11:26:18.453000+00:00,None,2020-07-30 11:26:18.453000+00:00,None,None,None,None,9666018,63159655,2,0,None,None
1,63172205,None,"<p>I know this is an old one, but here's a bit...",None,None,0,None,2020-07-30 11:28:08.737000+00:00,None,2020-07-30 11:28:08.737000+00:00,None,None,None,None,8452082,51420271,2,0,None,None
2,63172206,None,<p>You could try entering this into your URL b...,None,None,2,None,2020-07-30 11:28:11.363000+00:00,None,2020-07-30 11:28:11.363000+00:00,None,None,None,None,13665232,63169739,2,0,None,None
3,63172232,None,<blockquote>\n<p>Here you get simple spring bo...,None,None,0,None,2020-07-30 11:29:20.073000+00:00,None,2020-07-30 11:29:20.073000+00:00,None,None,None,None,5375134,22502400,2,0,None,None
4,63172234,None,"<pre class=""lang-py prettyprint-override""><cod...",None,None,2,None,2020-07-30 11:29:23.420000+00:00,None,2020-07-30 11:29:23.420000+00:00,None,None,None,None,2066113,63172053,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [7]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,60645095,Unable to successfully upload a file via Slack...,<p>I'm unable to successfully upload a file ev...,NaN,2,0,None,2020-03-11 22:14:06.287000+00:00,NaN,2020-03-26 04:50:05.180000+00:00,2020-03-12 13:00:13.987000+00:00,None,13048023.0,None,13048023,None,1,1,slack,256
1,60657533,How to dynamically pass the different time on ...,<p>Code :</p>\n\n<pre><code>class Schedule ext...,60657845.0,2,0,None,2020-03-12 15:47:07.380000+00:00,NaN,2020-03-13 14:15:57.113000+00:00,2020-03-12 15:50:17.007000+00:00,None,10834477.0,None,10834477,None,1,0,javascript|reactjs,256
2,60686369,Laravel AWS Elastic Beanslack deployment error...,<p>I deployed my Application in AWS Elastic Be...,NaN,1,0,None,2020-03-14 19:07:13.653000+00:00,NaN,2020-05-25 18:29:26.490000+00:00,NaT,None,NaN,None,12680939,None,1,0,laravel|amazon-web-services|deployment|amazon-...,256
3,60703409,Attempt to invoke virtual method 'void android...,"<p>so basically when I run my app, the app doe...",NaN,2,0,None,2020-03-16 09:33:09.347000+00:00,NaN,2020-03-16 11:03:40.043000+00:00,2020-03-16 11:03:40.043000+00:00,None,2649012.0,None,13069576,None,1,-1,java|android|android-recyclerview|nullpointere...,256
4,60713738,How to test react navigation authentication flow?,<p>I'm developing a React Native application. ...,60732455.0,2,0,None,2020-03-16 21:57:10.760000+00:00,1.0,2020-03-18 01:44:17.623000+00:00,2020-03-16 22:02:08.203000+00:00,None,781366.0,None,781366,None,1,0,typescript|react-native|jestjs|react-navigation,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [8]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [9]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  57871303  Can't get data from Google Pub/Sub to BigQuery...     12046719.0
1  57897905  running async JS functions on BigQuery with #s...       132438.0
2  57673421     Convert HTML characters to unicode in BigQuery      2021517.0
3  57887553  Insert timestamp into bigquery table using pandas      4423230.0
4  57723018  Connection::SQLGetInfoW: [Simba][ODBC] (11180)...      5293267.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [10]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [11]:
# Your code here
answers_query = """SELECT a.id, a.body, a.owner_user_id
FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
ON q.id = a.parent_id
WHERE q.tags LIKE '%bigquery%'
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query,job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

BadRequest: 400 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/a012e36a-a5e9-47b1-8ec6-443005ee1b92?maxResults=0&location=US: Query exceeded limit for bytes billed: 10000000000. 25831669760 or higher required.

## note ##

My answer is actually correct. I checked the solution and it is absolutely identical. I even copy-pasted the solution and ran it just to be sure and got the same error message.

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_4.hint()
q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [15]:
# Your code here
bigquery_experts_query = """
SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
    ON a.parent_id = q.id
WHERE q.tags LIKE '%bigquery%'
GROUP BY user_id


"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

     user_id  number_of_answers
0  9720191.0                  2
1  4788501.0                  1
2  9947159.0                  3
3  6885516.0                  3
4  8016720.0                 13


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [17]:
#q_5.hint()
#q_5.solution()

## note ##
I got everything correct on my first try - yay :)

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

# Your code here
requested_tag = get_requested_tag_from_website()
bigquery_experts_query = """
DECLARE @requested_tag string
SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
    ON a.parent_id = q.id
WHERE q.tags LIKE '%@requested_tag%'
GROUP BY user_id


"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?fileType=bigQuery).

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*